In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!pip install -q transformers torch librosa pydub nltk sentencepiece accelerate
!pip install -q git+https://github.com/openai/whisper.git


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 6.4 MB/s eta 0:00:00


In [ ]:
import whisper
import librosa
import nltk
import torch
import re
from transformers import pipeline
from nltk.tokenize import sent_tokenize


In [ ]:
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
def preprocess_audio(audio_path):
    audio, sr = librosa.load(audio_path, sr=16000)
    return audio


In [ ]:
asr_model = whisper.load_model("base")  # base = good accuracy + fast


100%|███████████████████████████████████████| 139M/139M [00:02<00:00, 64.5MiB/s]


In [ ]:
def transcribe_audio(audio_path):
    result = asr_model.transcribe(audio_path)
    return result["text"]


In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\b(uh|um|ah|er|mm)\b', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()


In [ ]:
def chunk_text(text, max_words=500):
    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = []

    word_count = 0
    for sentence in sentences:
        words = sentence.split()
        word_count += len(words)
        current_chunk.append(sentence)

        if word_count >= max_words:
            chunks.append(" ".join(current_chunk))
            current_chunk = []
            word_count = 0

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks


In [ ]:
summarizer = pipeline(
    "summarization",
    model="facebook/bart-large-cnn",
    device=0 if torch.cuda.is_available() else -1
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [ ]:
def summarize_chunks(chunks):
    summaries = []

    for chunk in chunks:
        summary = summarizer(
            chunk,
            max_length=150,
            min_length=50,
            do_sample=False
        )[0]['summary_text']

        summaries.append(summary)

    return summaries


In [ ]:
def final_summary(chunk_summaries):
    combined_text = " ".join(chunk_summaries)

    final = summarizer(
        combined_text,
        max_length=200,
        min_length=80,
        do_sample=False
    )[0]['summary_text']

    return final


In [ ]:
def podcast_summarizer(audio_path):
    print("🔊 Transcribing audio...")
    transcript = transcribe_audio(audio_path)

    print("🧹 Cleaning transcript...")
    clean_transcript = clean_text(transcript)

    print("✂️ Chunking text...")
    chunks = chunk_text(clean_transcript)

    print(f"📦 Total Chunks: {len(chunks)}")

    print("🧠 Summarizing chunks...")
    chunk_summaries = summarize_chunks(chunks)

    print("✨ Creating final summary...")
    final = final_summary(chunk_summaries)

    return {
        "transcript": clean_transcript,
        "chunk_summaries": chunk_summaries,
        "final_summary": final
    }


### Downloading a sample audio file

I'll download a sample audio file to demonstrate the `podcast_summarizer` function. If you have your own audio file, you can replace this with your file's path.

In [ ]:
# Download a sample audio file
!wget -O sample_audio.mp3 https://www.soundhelix.com/examples/mp3/SoundHelix-Song-1.mp3

audio_file_path = 'sample_audio.mp3'

print(f"Using audio file: {audio_file_path}")

--2026-01-19 15:46:50--  https://www.soundhelix.com/examples/mp3/SoundHelix-Song-1.mp3
Resolving www.soundhelix.com (www.soundhelix.com)... 81.169.145.157, 2a01:238:20a:202:1157::
Connecting to www.soundhelix.com (www.soundhelix.com)|81.169.145.157|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8945229 (8.5M) [audio/mpeg]
Saving to: ‘sample_audio.mp3’

sample_audio.mp3    100%[===================>]   8.53M  7.63MB/s    in 1.1s    

2026-01-19 15:46:52 (7.63 MB/s) - ‘sample_audio.mp3’ saved [8945229/8945229]

Using audio file: sample_audio.mp3


### Calling the `podcast_summarizer` function

In [ ]:
# Call the podcast_summarizer function again, now that NLTK resources are ready
summary_output = podcast_summarizer(audio_file_path)

print("\nFINAL PODCAST SUMMARY:")
print(summary_output["final_summary"])

🔊 Transcribing audio...


/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Your max_length is set to 150, but your input_length is only 9. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=4)


🧹 Cleaning transcript...
✂️ Chunking text...
📦 Total Chunks: 1
🧠 Summarizing chunks...


Your max_length is set to 200, but your input_length is only 50. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=25)


✨ Creating final summary...

FINAL PODCAST SUMMARY:
CNN.com will feature iReporter photos in a weekly Travel Snapshots gallery. Please submit your best shots of New York for next week. Visit CNN.com/Travel next Wednesday for a new gallery of snapshots from around the world. Click through the gallery to see more from the week's Travel Snapshot gallery. Click here to see the gallery from the previous week. The gallery will be updated every Wednesday.


In [ ]:
!pip install nltk

In [ ]:
audio_file_path = 'sample_audio.mp3'
print(f"The audio file path is: {audio_file_path}")

# Example of how you might open and read an audio file (this is illustrative and might need specific libraries for audio):
# with open(audio_file_path, 'rb') as f:
#     # Read some content or pass to an audio processing library
#     print(f"Successfully opened {audio_file_path}")

The audio file path is: sample_audio.mp3


In [ ]:
df.to_csv("/mnt/data/podcast_summary_results.csv", index=False)


NameError: name 'df' is not defined

In [ ]:
import pandas as pd

# Convert the summary_output dictionary to a DataFrame
# Since chunk_summaries is a list, we'll store it as a list in the DataFrame cell
df = pd.DataFrame({
    'transcript': [summary_output['transcript']],
    'chunk_summaries': [summary_output['chunk_summaries']],
    'final_summary': [summary_output['final_summary']]
})

# Display the DataFrame
display(df)

,transcript,chunk_summaries,final_summary
0,"man, this is on too!",[CNN.com will feature iReporter photos in a we...,CNN.com will feature iReporter photos in a wee...


In [ ]:
print("FINAL PODCAST SUMMARY:")
print(df['final_summary'].iloc[0])

FINAL PODCAST SUMMARY:
CNN.com will feature iReporter photos in a weekly Travel Snapshots gallery. Please submit your best shots of New York for next week. Visit CNN.com/Travel next Wednesday for a new gallery of snapshots from around the world. Click through the gallery to see more from the week's Travel Snapshot gallery. Click here to see the gallery from the previous week. The gallery will be updated every Wednesday.


In [ ]:
df.to_csv('podcast_summary_results.csv', index=False)
print("DataFrame 'df' saved to 'podcast_summary_results.csv'")

DataFrame 'df' saved to 'podcast_summary_results.csv'


In [ ]:
import pandas as pd

df = pd.read_csv("/content/podcast_summary_results.csv")


In [ ]:
df.columns


Index(['transcript', 'chunk_summaries', 'final_summary'], dtype='object')

In [ ]:
Index(['Podcast_Name', 'Episode_Title', 'Episode_Length_minutes',
       'Genre', 'Host_Popularity_percentage', 'Publication_Day',
       'Publication_Time', 'Guest_Popularity_percentage',
       'Number_of_Ads', 'Episode_Sentiment', 'Listening_Time_minutes'],
      dtype='object')



NameError: name 'Index' is not defined

In [ ]:
display(df.head())

,transcript,chunk_summaries,final_summary
0,"man, this is on too!",['CNN.com will feature iReporter photos in a w...,CNN.com will feature iReporter photos in a wee...


In [ ]:
display(df.head())

,transcript,chunk_summaries,final_summary
0,"man, this is on too!",['CNN.com will feature iReporter photos in a w...,CNN.com will feature iReporter photos in a wee...


In [ ]:
Index(['Podcast_Name', 'Episode_Title', 'Episode_Length_minutes',
       'Genre', 'Host_Popularity_percentage', 'Publication_Day',
       'Publication_Time', 'Guest_Popularity_percentage',
       'Number_of_Ads', 'Episode_Sentiment', 'Listening_Time_minutes'],
      dtype='object')


NameError: name 'Index' is not defined

In [ ]:
print(df.columns)

Index(['transcript', 'chunk_summaries', 'final_summary'], dtype='object')


In [ ]:
df[['Podcast_Name', 'Episode_Title']].head(10)


KeyError: "None of [Index(['Podcast_Name', 'Episode_Title'], dtype='object')] are in the [columns]"

In [ ]:
print(df.columns)

Index(['transcript', 'chunk_summaries', 'final_summary'], dtype='object')


In [ ]:
df[['Podcast_Name', 'Episode_Title']].head(10)


KeyError: "None of [Index(['Podcast_Name', 'Episode_Title'], dtype='object')] are in the [columns]"

In [ ]:
display(df[['transcript', 'chunk_summaries', 'final_summary']])

,transcript,chunk_summaries,final_summary
0,"man, this is on too!",['CNN.com will feature iReporter photos in a w...,CNN.com will feature iReporter photos in a wee...
